# Header

ใน workbook นี้จะเป็น experimental code ก่อนที่จะไป deploy UI บน huggingface ครับ โดยจะประกอบไปด้วย 4 ส่วนหลักๆคือ

**1.** Encoding

**2.** Upload ข้อมูลขึ้น Qdrant database

**3.** RAG & Prompt Engineering

**4.** RAG + Text generation model

## 1. Encoding

ในส่วนนี้จะดึงข้อมูลจากไฟล์ CSV ที่ได้มากจาก web scraping นำข้อมูลมา encode โดย sentence transformer model

### Import Env Variable

In [134]:
import os
from dotenv import load_dotenv

In [135]:
# Load environment variables from the .env file
load_dotenv()

# Access API
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_TOKEN")
QROQ_API_KEY = os.getenv("QROQ_TOKEN")

#Langsmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["GROQ_API_KEY"] = QROQ_API_KEY

QDRANT_CLOUD_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_TOKEN")

OPENAI_API_KEY = os.getenv("OPENAI_TOKEN")

### Encoding

ในส่วนนี้จะทำการ encode ทั้งคำถามและคำตอบจากกระทู้ โดยใช้ model **paraphrase-multilingual-mpnet-base-v2** ซึ่งเป็น model ที่รองรับภาษาไทย การ encode จะทำการ encode ทั้งคำถามและคำตอบแยกกัน เพื่อเอาไว้ใช้งานในวิธี **Two-Stage Retrieval** ซึ่งจะอธิบายต่อไป

In [127]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

In [128]:
# Load your data
file_path = "Agnos_Healthcare.csv"
data = pd.read_csv(file_path)

# Initialize the sentence transformer model
embedding_model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

# Generate embeddings for the questions
data['Question_embedding'] = data['Question'].apply(lambda x: embedding_model.encode(x).tolist())

# Generate embeddings for the answers
data['Answer_embedding'] = data['Answer'].apply(lambda x: embedding_model.encode(x).tolist())


In [129]:
data.head()

,url,Question,Answer,Question_embedding,Answer_embedding
0,https://www.agnoshealth.com/forums/%E0%B8%A5%E...,ไปพบแพทย์ ฉีดยาแล้ว ก็พอทุเลาลงไปบ้าง แต่พออีก...,ยาที่กล่าวมาค่อนข้างตรงกับอาการที่เป็นอยู่แล้ว...,"[0.03410579264163971, -0.04926569387316704, -0...","[0.031183620914816856, -0.04248825088143349, -..."
1,https://www.agnoshealth.com/forums/%E0%B8%A0%E...,สวัสดีค่ะ อยากทราบว่าอาการคิดมาก อยากร้องไห้ตล...,สวัสดีครับ การที่อยากคิดมาก ร้องไห้ตลอด อยากอย...,"[-0.019984273239970207, 0.10949759185314178, -...","[-0.04914068058133125, -0.005477641709148884, ..."
2,https://www.agnoshealth.com/forums/%E0%B9%84%E...,ผายลมทั้งวัน เกิดจากสาเหตุอะไรคะ แล้วผิดปกติรึ...,แปลว่ามีลมในทางเดินอาหารเยอะ สาเหตุได้หลายอย่า...,"[-0.05385837331414223, -0.058753013610839844, ...","[0.03273347392678261, -0.049386586993932724, -..."
3,https://www.agnoshealth.com/forums/%E0%B8%81%E...,สวัสดีครับ ผมเหมือนจะเป็นกระเพาะปัสสาวะอักเสบน...,สวัสดีครับ จากอาการที่แจ้ง คือปัสสาวะบ่อย กะปร...,"[0.08119186013936996, 0.023244842886924744, -0...","[0.050747279077768326, -0.010291456244885921, ..."
4,https://www.agnoshealth.com/forums/%E0%B8%A0%E...,ตกขาวปนเลือดหลังจากประจำเดือนหมด,จากประวัติคนไข้ อาการตกขาวผิดปกติร่วมกับอาการค...,"[0.040805839002132416, -0.04118615388870239, -...","[0.032024502754211426, 0.009404908865690231, -..."


## 2. Upload to Qdrant database

จะทำการ upload ข้อมูลขึ้นไปบน Qdrant database โดยข้อมูลแต่ละ Payload จะประกอบด้วย 
```sh
{
    payload :   {
                    question : "ไปพบแพทย์ ฉีดยาแล้ว ก็พอทุเลาลงไปบ้าง", #คำถามจากกระทู้
                    answer : "ยาที่กล่าวมาค่อนข้างตรงกับอาการที่เป็นอยู่", #คำตอบจากคุณหมอ
                    type : "question" #ประเภทการ encode (question หมายถึงใช้คำถามไป encode, answer หมายถึงใช้คำตอบไป encode)
                }

    vector : [1,2,3,4,...]
}

```

### Create collection

In [75]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.http.models import PointStruct

In [76]:
# Initialize Qdrant client with your cloud credentials
qdrant_client = QdrantClient(
    url=QDRANT_CLOUD_URL,  
    api_key=QDRANT_API_KEY           
)

# Define the collection name
collection_name = "Agnos_collection"

# Create a collection in Qdrant
qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=data['Question_embedding'][0].__len__(), distance=Distance.COSINE)
)


True

### Create Payload & Upload Data

In [77]:
# Prepare payloads
payloads = [
    PointStruct(
        id=i, 
        vector=question_embedding,  # First embedding (Question)
        payload={"question": question, "answer": answer, "type": "question"}
    )
    for i, (question, answer, question_embedding) in enumerate(zip(data['Question'], data['Answer'], data['Question_embedding']))
] + [
    PointStruct(
        id=len(data['Question']) + i, 
        vector=answer_embedding,  # Second embedding (Answer)
        payload={"question": question, "answer": answer, "type": "answer"}
    )
    for i, (question, answer, answer_embedding) in enumerate(zip(data['Question'], data['Answer'], data['Answer_embedding']))
]

# Upload to Qdrant
batch_size = 10

for i in tqdm(range(0, len(payloads), batch_size), desc="Uploading batches"):
    batch = payloads[i:i + batch_size]
    qdrant_client.upsert(collection_name=collection_name, points=batch)


Uploading batches: 100%|██████████| 113/113 [01:26<00:00,  1.30it/s]


## 3. RAG & Prompt Engineering

### RAG

จะใช้ RAG แบบ 2 stage retrieval คือ 

**1.** เมื่อมี query ใหม่เข้ามาจากผู้ใช้งาน จะหาข้อมูลจาก **คำถาม** ที่เกี่ยวข้อมากที่สุดก่อน ว่าคำถามนี้เคยถูกถามมาหรือไม่ 

**2.** จะคิดค่า similarity ซึ่งเปรียบเสมือนค่าความมั่นใจในการตัดสินใจว่า query ใหม่กับคำถามที่เคยถูกถามในกระทู้ มีความเหมือนกันมากน้อยเพียงใด 

**3.** ถ้าค่า similarity >= 75% จะดึงข้อมูล **คำตอบ** จากหมอมาใช้งานต่อไป

**4.** ถ้าค่า similarity < 75% จะหาข้อมูลจาก **คำตอบ** ที่เกี่ยวข้อมากที่สุดก่อน ว่าคำถามนี้เคยถูกตอบมาหรือไม่

**5.** นำผลลัพท์เอกสารที่เกี่ยวข้องที่ได้ทั้งหมด ไปทำการ **rerank** โดยใช้ BM25
เพื่อหาค่าความคล้ายคลึงอีกครั้งและเรียงลำดับก่อนที่จะป้อนให้กับ text generation model ต่อไป

In [117]:
from langchain_qdrant import QdrantVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
from qdrant_client.http.models import FieldCondition, Filter, MatchValue, models
from rank_bm25 import BM25Okapi
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [112]:
# Use the same embedding model
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

# Connect LangChain to the Qdrant collection
vectorstore = QdrantVectorStore(
    client=qdrant_client,
    collection_name=collection_name,
    embedding=embeddings,
    content_payload_key="answer"
)


In [131]:
def retrieve_context(query, k=3, threshold=0.75):
    """Retrieve the most relevant answer using two-stage search."""
    
    query_vector = embeddings.embed_query(query)  # Embed the user query
    
    # Step 1: Search for similar QUESTIONS only
    question_filter = Filter(
        must=[
            FieldCondition(
                key="type",
                match=MatchValue(value="question")
            )
        ]
    )
    search_results = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_vector,
        limit=k,
        query_filter=question_filter  # Filter only questions
    )

    # Extract answers from matched questions
    answers = [result.payload["answer"] for result in search_results]

    # If high-confidence matches found, rerank and return them
    # If no strong match, search for similar ANSWERS
    if search_results and search_results[0].score < threshold:
    
        # Step 2: If no strong match, search for similar ANSWERS directly
        answer_filter = Filter(
            must=[
                FieldCondition(
                    key="type",
                    match=MatchValue(value="answer")
                )
            ]
        )
        
        fallback_results = qdrant_client.search(
            collection_name=collection_name,
            query_vector=query_vector,
            limit=k,
            query_filter=answer_filter  # Now filter only answers
        )

        # Extract answers from matched answers
        answers = [result.payload["answer"] for result in fallback_results]

    # rerank
    bm25 = BM25Okapi(answers)
    bm25_scores = bm25.get_scores(query.split())
    ranked_results = sorted(
        zip(search_results, bm25_scores),
        key=lambda x: x[1], reverse=True
    )
    
    return ranked_results

In [169]:
# Test the function
query = "ปวดท้อง ทำยังไงดีคะ"
context = retrieve_context(query)
context

[(ScoredPoint(id=111, version=11, score=0.8957808, payload={'question': 'มันปวดท้องขึ้นเรื่อยๆเลยค่ะทำไงดีคะ', 'answer': 'อาการถ่ายเหลวปนมูก ร่วมกับปวดบิดท้อง อาเจียน เข้าได้กับภาวะลำไส้อักเสบติดเชื้อครับ การรักษาหลักจะเป็นการทานยาฆ่าเชื้อ เช่น Ciprofloxacin, Cefixime นาน 5 วัน ร่วมกับการทานยาตามอาการ เช่น ยาแก้ปวดบิดท้อง Hyoscine, ยาแก้อาเจียน Domperidone เป็นต้น อาการจะดีขึ้นใน 2-3 วัน ครับ', 'type': 'question'}, vector=None, shard_key=None, order_value=None),
  0.0),
 (ScoredPoint(id=470, version=47, score=0.8473103, payload={'question': 'คือผมคลื่นไส้ทุกวันและปวดท้องบริเวณสะดือทรมานมากควรทำำงดีครับหมอ', 'answer': 'อาการปวดท้องบริเวณสะดือ ร่วมกับคลื่นไส้อาเจียน ส่วนใหญ่เกิดจากอาหารเป็นพิษ หรือลำไส้อักเสบ สามารถหายได้เองใน 2-3 วัน การรักษาหลักจะเป็นการทานยาตามอาการ เช่น ยาแก้คลื่นไส้ (Domperidone, Metoclopramide) ยาแก้ปวดบิดท้อง (Hyoscine) และทานผงเกลือแร่ ORS จิบบ่อยๆทั้งวัน ก็จะช่วยให้อาการหายเร็วขึ้นได้ครับ แต่ถ้าอาการเป็นเรื้อรังนานกว่า 5-7 วัน ควรไปตรวจเพิ่มเติมที่โรงพยาบาล อาจม

### Prompt Engineering

credit : [สร้าง AI ตอบคำถามด้วย LLaMA3.1, Langchain, RAG และ FAISS](https://blog.appsynth.net/%E0%B8%AA%E0%B8%A3%E0%B9%89%E0%B8%B2%E0%B8%87-ai-%E0%B8%95%E0%B8%AD%E0%B8%9A%E0%B8%84%E0%B8%B3%E0%B8%96%E0%B8%B2%E0%B8%A1%E0%B8%94%E0%B9%89%E0%B8%A7%E0%B8%A2-llama3-1-langchain-rag-%E0%B9%81%E0%B8%A5%E0%B8%B0-faiss-cf8e57ac5a5d) by [nutron](https://medium.com/@nutron)

ในส่วนนี้ จะรวม query ที่ผู้ใช้งานได้ป้อนเข้ามาใหม่กับการระบุบทบาทให้กับ chatbot และบริบท เพื่อการตอบคำถามได้ดียิ่งขึ้น

In [154]:
from langchain_core.runnables import Runnable
class ChatGPTPromptRunnable(Runnable):
    def __init__(self, system=""):
        super().__init__()
        self.system = system

    def invoke(self, inputs: dict, config=None) -> str:
        question = inputs["question"]
        context = inputs["context"]
        # Create the system prompt if provided
        system_prompt = ""
        if self.system != "":
            system_prompt = (
                f"<|start_header_id|>system<|end_header_id|>\n\n{self.system}\n\n"
                f"context: {context}\n\n"
                f"<|eot_id|>\n\n"
            )
            prompt = (
                f"<|begin_of_text|>{system_prompt}"
                f"<|start_header_id|>user<|end_header_id|>\n\n"
                f"{question}\n\n"
                f"<|eot_id|>\n\n"
                f"<|start_header_id|>assistant<|end_header_id|>\n\n" # header - assistant
            )

        # Return the formatted prompt
        return prompt

In [241]:
# Example usage
gpt_prompt = ChatGPTPromptRunnable(system="คุณคือผู้ช่วยตอบคำถามสุขภาพ ฉันจะให้ข้อมูลเกี่ยวกับปัญหาสุขภาพที่เคยถูกถามและตอบโดยคุณหมอ ในกระทู้ของ Agnos คุณสามารถใช้เป็นแหล่งอ้างอิงได้")
print(gpt_prompt.invoke({"context": "ตอบคำถามสุขภาพ", "question": "ปวดท้องน้อย เป็นๆหายๆ ร่วมกับประจำเดือนมาช้าค่ะ เป็นโรคอะไรได้บ้างคะ"}))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

คุณคือผู้ช่วยตอบคำถามสุขภาพ ฉันจะให้ข้อมูลเกี่ยวกับปัญหาสุขภาพที่เคยถูกถามและตอบโดยคุณหมอ ในกระทู้ของ Agnos คุณสามารถใช้เป็นแหล่งอ้างอิงได้

context: ตอบคำถามสุขภาพ

<|eot_id|>

<|start_header_id|>user<|end_header_id|>

ปวดท้องน้อย เป็นๆหายๆ ร่วมกับประจำเดือนมาช้าค่ะ เป็นโรคอะไรได้บ้างคะ

<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>




## 4. RAG + Text generation model

ในส่วนนี้จะทำการต่อ RAG เข้ากับ LLM text generation model เพื่อให้ chatbot สามารถตอบโต้ได้โดยอาศัยความรู้ที่เราป้อนให้ โดยที่จะใช้ model gpt-4o-mini ของ OpenAI เป็น text generation model และยังมีการทำ Chain conversation โดยใช้ Langchain เพื่อให้ chatbot ไม่ลืมบริบทก่อนหน้า 

In [236]:
from langchain.schema import BaseRetriever
from langchain_openai import ChatOpenAI
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
from langchain_core.documents import Document
from pydantic import Field
from typing import List

# Initialize LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=OPENAI_API_KEY)

class CustomRetriever(BaseRetriever):
    """Custom retriever that integrates `retrieve_context` into LangChain while maintaining query history."""

    query_context: str = Field(default="")  # Define it as a Pydantic field

    def get_relevant_documents(self, query: str) -> List[Document]:
        # Append new query to maintain context
        self.query_context += " " + query  
        
        ranked_results = retrieve_context(self.query_context, k=3, threshold=0.75)
        
        return [
            Document(page_content=result[0].payload["answer"], metadata={"score": result[0].score})
            for result in ranked_results
        ]

    async def aget_relevant_documents(self, query: str) -> List[Document]:
        return self.get_relevant_documents(query)  # Sync fallback for async

# Use CustomRetriever
custom_retriever = CustomRetriever(query_context="")

# Use the custom retriever in the chain
chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=custom_retriever,
    return_source_documents=True
)

chat_history = []

query = "ปวดท้องน้อย เป็นๆหายๆ ร่วมกับประจำเดือนมาช้าค่ะ เป็นโรคอะไรได้บ้างคะ"
response = chain.invoke({"question": query, "chat_history": chat_history})

print(response['answer'])


อาการปวดท้องน้อยเป็นๆ หายๆ ร่วมกับประจำเดือนมาช้า อาจมีสาเหตุมาจากหลายปัจจัย รวมถึงความผิดปกติของมดลูก เช่น เนื้องอกมดลูกหรือความไม่สมดุลของฮอร์โมนเพศ แนะนำให้ไปพบสูตินรีแพทย์เพื่อตรวจสอบและวินิจฉัยอย่างละเอียด จะต้องมีการตรวจภายในหรืออัลตราซาวด์เพื่อหาสาเหตุที่แน่นอนค่ะ


เรียกใช้ครั้งที่ 2 จะเห็นว่า chatbot ยังไม่ลืมบริบทที่คุยกันก่อนหน้า

In [237]:
chat_history = [(query, response["answer"])]

query = "มีแนวทางการดูแลตนเองยังไงบ้างคะ"
result = chain({"question": query, "chat_history": chat_history})

print(result['answer'])

เกี่ยวกับการดูแลตนเองในกรณีที่มีอาการเกี่ยวกับประจำเดือนหรืออาการปวดท้อง หากอาการไม่รุนแรง สามารถทำตามแนวทางดังนี้:

1. รักษาความสะอาด: ควรดูแลความสะอาดในช่วงมีประจำเดือน เพื่อป้องกันการติดเชื้อ
2. ออกกำลังกาย: การเคลื่อนไหวร่างกายเบา ๆ อาจช่วยลดอาการปวดท้องได้
3. ดูแลโภชนาการ: ทานอาหารที่มีประโยชน์ ผักผลไม้ และดื่มน้ำให้เพียงพอ
4. ใช้ยาเบื้องต้น: หากมีอาการปวด สามารถพิจารณาทานยาแก้ปวดที่ไม่ต้องสั่งโดยแพทย์ ตามที่เภสัชกรแนะนำ
5. หลีกเลี่ยงความเครียด: พยายามหากิจกรรมที่ช่วยให้ผ่อนคลาย เช่น การทำสมาธิหรือโยคะ

แต่ถ้าอาการไม่ดีขึ้น หรือมีอาการรุนแรง ควรไปพบแพทย์เพื่อตรวจเพิ่มเติมครับ


พร้อมสำหรับการทำ UI ขึ้น huggingface🤗 แล้วครับ